In [2]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
#from census import Census
import gmaps
from scipy.stats import linregress
import numpy as np
import os
from apikeys import gkey


In [3]:
# Load the combined CSV file
combined_df = pd.read_csv("combined.csv")

# Display sample data
combined_df

,Unnamed: 0,State,Name,Lat,Lng,Population,Median Age,Household Income,Per Capita Income,Poverty Count,...,owned_households_%,rented_households_%,homes_#_powered_by_solar,homes_%_powered_by_solar,state_elect_from_solar_%,solar_companies_in_state,MW_currently_installed,MW_growth_projection_5_years,MW_total_in_5_years,MW_output_%_in_5_years
0,12,1,Alabama,32.3774,-86.3005,4893186,39.2,52035,28934,762642,...,70.0,30.0,61870,3.356899,0.0052,35,578,2393,2971,514.013841
1,16,4,Arizona,33.4481,-112.0970,7174064,37.9,61529,32340,990528,...,67.6,32.4,945767,39.447639,0.0970,296,5984,5342,11326,189.271390
2,34,5,Arkansas,34.7470,-92.2891,3011873,38.3,49475,27724,470190,...,67.0,33.0,65256,6.899403,0.0122,27,588,2533,3121,530.782313
3,1,6,California,38.5766,-121.4932,39346023,36.7,78672,38576,4853434,...,55.9,44.1,10133299,80.794535,26.2100,1977,37086,24874,61960,167.071132
4,28,8,Colorado,39.7393,-104.9848,5684926,36.9,75231,39545,544232,...,66.8,33.2,440055,21.927047,0.0595,330,2268,3946,6214,273.985891
5,20,9,Connecticut,41.7658,-72.6734,3570549,41.1,79855,45668,339156,...,66.6,33.4,165593,12.307730,0.0302,135,1137,1011,2148,188.918206
6,47,10,Delaware,39.1573,-75.5197,967679,41.0,69110,36574,107641,...,72.6,27.4,19742,6.451444,0.0459,37,172,508,680,395.348837
7,5,11,District of Columbia,38.8899,-77.0091,701974,34.1,90842,58659,103391,...,41.6,58.4,28186,11.738786,0.4948,157,184,267,451,245.108696
8,7,12,Florida,30.4381,-84.2826,21216924,42.2,57703,32848,2772939,...,67.4,32.6,1144171,15.739379,0.0499,378,9539,10935,20474,214.634658
9,11,13,Georgia,33.7492,-84.3881,10516579,36.9,61224,32427,1461572,...,66.0,34.0,527774,14.882766,0.0469,176,4566,1761,6327,138.567674


In [6]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Store the latitude and longitude in locations and store the humidity to be used as the weight
locations = combined_df[["Lat", "Lng"]].astype(float)
futureIncrease = combined_df["MW_growth_projection_5_years"].astype(float)
radiation = combined_df["daily_solar_radiation"].astype(float)

In [7]:
# Plot Heatmap 
fig = gmaps.figure()

# Create heat layer
future_layer = gmaps.heatmap_layer(locations, weights = futureIncrease, dissipating = False, 
                                 max_intensity = max(futureIncrease), point_radius = 4)

# Add heat layer 
fig.add_layer(future_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Narrow down cities for ideal weather conditions and drop any null values
ideal_solar = combined_df.loc[(combined_df["daily_solar_radiation"] >= 4) & (combined_df["electricity_price_residential"] >= 12) & (combined_df["pop_density"] <= 80)].dropna()

# Reset the index
ideal_solar = ideal_solar.reset_index(drop = True)

# Display results
ideal_solar

,Unnamed: 0,State,Name,Lat,Lng,Population,Median Age,Household Income,Per Capita Income,Poverty Count,...,owned_households_%,rented_households_%,homes_#_powered_by_solar,homes_%_powered_by_solar,state_elect_from_solar_%,solar_companies_in_state,MW_currently_installed,MW_growth_projection_5_years,MW_total_in_5_years,MW_output_%_in_5_years
0,16,4,Arizona,33.4481,-112.0970,7174064,37.9,61529,32340,990528,...,67.6,32.4,945767,39.447639,0.0970,296,5984,5342,11326,189.271390
1,28,8,Colorado,39.7393,-104.9848,5684926,36.9,75231,39545,544232,...,66.8,33.2,440055,21.927047,0.0595,330,2268,3946,6214,273.985891
2,36,20,Kansas,39.0483,-95.6781,2912619,36.9,61091,32798,323644,...,67.0,33.0,15102,1.710961,0.0022,42,106,1308,1414,1333.962264
3,46,32,Nevada,39.1641,-119.7661,3030281,38.2,62043,32629,381695,...,59.1,40.9,857303,113.331209,0.2187,87,5003,7707,12710,254.047572
4,45,35,New Mexico,35.6824,-105.9399,2097021,38.1,51243,27945,381026,...,69.5,30.5,344556,45.448622,0.0605,69,1452,4365,5817,400.619835


In [9]:
# Plot Heatmap 
fig = gmaps.figure()

# Create heat layer
ideal_solar = gmaps.heatmap_layer(locations, weights = radiation, dissipating = False, 
                                 max_intensity = max(radiation), point_radius = 4)

# Add heat layer 
fig.add_layer(ideal_solar)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))